# Unstructured grid

## Reading data from a mesh.

Here we read the data from the ocean model MIT/GCM/LLC4320

In [ ]:
import intake

cat_url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml"
cat = intake.Catalog(cat_url)

Reading of the grid positions. The grid is under-sampled because the volume of data is very large.

In [ ]:
grid = cat.LLC4320_grid.to_dask()
grid

In [ ]:
lons = grid["XC"].isel(i=slice(0, None, 8), j=slice(0, None, 8)).values
lats = grid["YC"].isel(i=slice(0, None, 8), j=slice(0, None, 8)).values

In [ ]:
del grid

Reading data. We get one time frame only.

In [ ]:
ssh = cat.LLC4320_SSH.to_dask()
ssh = ssh["Eta"].isel(time=0, i=slice(0, None, 8), j=slice(0, None, 8)).values

## R\*Tree

The interpolation of this object is based on an [R\*Tree](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree) structure. To begin with, we start by building this object. By default, this object considers WGS-84 geodetic coordinate system. But you can define another one using class [System](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.geodetic.html#pyinterp.geodetic.System).

In [ ]:
import pyinterp.rtree
mesh = pyinterp.rtree.RTree()

## Creating the search tree

Then, we will insert points into the tree. The class allows you to insert points using two algorithms. The first one called [packing](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.packing) allows you to insert the values in the tree at once. This mechanism is the recommended solution to create an optimized in-memory structure, both in terms of construction time and queries. When this is not possible, you can insert new information into the tree as you go along using the [insert](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.insert) method.

In [ ]:
import numpy as  np

mesh.packing(
    np.vstack((lons.flatten(), lats.flatten())).T,
    ssh.flatten())

When the tree is created, you can [interpolate](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.inverse_distance_weighting) the data or make various [queries](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.query) on the tree.

In [ ]:
mx, my = np.meshgrid(
    np.arange(-180, 180, 0.25),
    np.arange(-90, 90, 0.25),
    indexing="ij")

mss, neighbors = mesh.inverse_distance_weighting(
    np.vstack((mx.flatten(), my.flatten())).T,
    within=False,
    radius=35434,
    k=8,
    num_threads=0)

Let's visualize our interpolated data

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.pcolormesh(mx, my, mss.reshape(mx.shape), cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Eta")